In [1]:
import copy
import gym
import os
import sys
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from skimage.transform import resize
from tensorflow.keras.layers import Input,Dense,Conv2D,Flatten
from tensorflow.keras import Model

In [2]:
MAX_EXPERIENCES = 500000
MIN_EXPERIENCES = 50000
TARGET_UPDATE_PERIOD = 10000
IM_SIZE = 84
K = 4 #env.action_space.n

In [3]:
class ImageTransformer:
    # transfrom image from 210,160,3
    # to 84,84
    def transform(self,state):
        state = tf.image.rgb_to_grayscale(state)
        state = tf.image.crop_to_bounding_box(state,34, 0, 160, 160)
        state = tf.image.resize(state,[IM_SIZE, IM_SIZE],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        state = tf.squeeze(state)
        return state


In [4]:
def update_state(state, obs_small):
    # throw oldest frame
    # append newest to end
    return np.append(state[:,:,1:],np.expand_dims(obs_small,2),axis=2)
    

In [5]:
class ReplayMemory:
    def __init__(self,size=MAX_EXPERIENCES,frame_height=IM_SIZE,frame_width = IM_SIZE,agent_history_length=4,batch_size=32):
        self.size =size
        self.frame_height = frame_height
        self.frame_width = frame_width
        self.agent_history_length = agent_history_length
        self.batch_size = batch_size
        self.count = 0
        self.current = 0
        
        self.actions = np.empty(self.size,dtype=np.int32)
        self.rewards = np.empty(self.size,dtype=np.float32)
        self.frames = np.empty((self.size,self.frame_height,self.frame_width),dtype=np.uint8)
        self.terminal_flags = np.empty(self.size,dtype=np.bool)
        # pre-allocate memory for batch
        self.states = np.empty((self.batch_size,self.agent_history_length,self.frame_height,self.frame_width),dtype=np.uint8) 
        self.new_states = np.empty((self.batch_size,self.agent_history_length,self.frame_height,self.frame_width),dtype=np.uint8) 
        self.indices = np.empty(self.batch_size,dtype=np.int32)
        
    def add_experience(self,action,frame,reward,terminal):
        self.actions[self.current] = action
        self.frames[self.current] = frame
        self.rewards[self.current] = reward
        self.terminal_flags[self.current] = terminal
        self.count = max(self.count,self.current+1)
        self.current = (self.current+1) % self.size
        
    def _get_state(self,index):
        #frames [t-3,t+1[
        return self.frames[index-self.agent_history_length+1:index+1, ...]
    
    def _get_valid_indices(self):
        for i in range(self.batch_size):
            while True:
                # start at agent_history_length since when sampling , 
                # this frame is taken along the previous agent_history_length-1 frames
                index = random.randint(self.agent_history_length, self.count - 1)
                # current is a circular index
                # reaching here means current made a cycle (see final comment)
                # in which case index = current-1 is newest experience , while index = current is oldest
                # thus having experinces before and after index is invalid since these are non-consecutive
                # if >= current was empty index would be less than current since count be current+1
                
                if index >= self.current and index - self.agent_history_length <= self.current:
                    continue
                # same goes here
                # if any of the experiences mark the end of the episode , the next experience is surely of a new episode
                # thus these are also non-consecutive experiences
                if self.terminal_flags[index - self.agent_history_length:index].any():
                    continue
                break
            self.indices[i] = index
          
    def get_minibatch(self):
        self._get_valid_indices()
        
        for i, idx in enumerate(self.indices):
            self.states[i] = self._get_state(idx - 1) # s (4 consecutive frames , s is last)
            self.new_states[i] = self._get_state(idx) # s'
        # states : N,T,H,W ----> N,H,W,T so it suits tensorflow , T acts like channels
        return np.transpose(self.states, axes=(0, 2, 3, 1)), self.actions[self.indices], self.rewards[self.indices], np.transpose(self.new_states, axes=(0, 2, 3, 1)), self.terminal_flags[self.indices]



In [6]:
def loss(targets,actions,yhat):
    selected_action_values = tf.reduce_sum(yhat * tf.one_hot(actions, K),axis=1)
    #cost = tf.reduce_mean(tf.square(targets - selected_action_values))
    cost = tf.reduce_mean(tf.keras.losses.Huber()(targets, selected_action_values))
    return cost

In [7]:
@tf.function
def train_step(model,opt,inputs,actions,targets):
    with tf.GradientTape() as tape:
        yhat = model(inputs, training=True)
        loss_value = loss(targets,actions,yhat)
    grads = tape.gradient(loss_value, model.trainable_weights)
    opt.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

In [8]:
def DQN(K,conv_layer_sizes,hidden_layer_sizes): # returns the DQN model
    input_ = Input(shape=(IM_SIZE,IM_SIZE,4))
    x = input_/255.0
    for num_output_filters,filtersz,poolsz in conv_layer_sizes:
        x = Conv2D(num_output_filters,filtersz,poolsz,activation='relu')(x)
    x = Flatten()(x)   
    for M in hidden_layer_sizes:
        x = Dense(M,activation='relu')(x)
    output = Dense(K,activation='relu')(x)
    model = Model(inputs=input_,outputs=output)
    return model

In [9]:
def copy_weights(from_,to):
    weights = [w.numpy() for w in from_.weights]
    to.set_weights(weights)

In [10]:
def sample_action(model,x,eps):
    if np.random.random() < eps:
        return np.random.choice(K)
    else:
        x = np.expand_dims(x,axis=0)
        return np.argmax(model(x).numpy()[0])


In [11]:
def train(model,opt,target_model,rb,gamma,batch_size):
    states, actions, rewards, next_states, dones = rb.get_minibatch()
    next_Qs = target_model.predict(next_states)
    next_Q = np.max(next_Qs,axis=1)
    targets = rewards + np.invert(dones).astype(np.float32) * gamma * next_Q
    loss = train_step(model,opt,states,actions,targets)
    

In [12]:
def play_one(env,total_t,rb,model,opt,target_model,im_transformer,gamma,batch_size,epsilon,epsilon_change,epsilon_min):
    t0 = datetime.now()
    obs = env.reset()
    obs_small = im_transformer.transform(obs)
    state = np.stack([obs_small]*4,axis=2)
    loss = None
    
    total_time_training = 0
    num_steps_in_episode = 0
    episode_reward = 0
    
    done = False
    while not done:
        if total_t % TARGET_UPDATE_PERIOD == 0:
            copy_weights(model,target_model)
            print("Copied model parameters to target network. total_t = %s, period = %s" % (total_t, TARGET_UPDATE_PERIOD))
            
        action = sample_action(model,state,epsilon)
        obs,reward,done,_ = env.step(action)
        obs_small = im_transformer.transform(obs)
        next_state = update_state(state,obs_small)

        episode_reward += reward
        rb.add_experience(action, obs_small, reward, done)

        t0_2 = datetime.now()
        loss = train(model, opt,target_model, rb, gamma, batch_size)
        dt = datetime.now() - t0_2

        # More debugging info
        total_time_training += dt.total_seconds()
        num_steps_in_episode += 1

        state = next_state
        total_t += 1

        epsilon = max(epsilon - epsilon_change, epsilon_min)

    return total_t, episode_reward, (datetime.now() - t0), num_steps_in_episode, total_time_training/num_steps_in_episode, epsilon


In [13]:
conv_layer_sizes = [(32, 8, 4), (64, 4, 2), (64, 3, 1)]
hidden_layer_sizes = [512]
gamma = 0.99
batch_sz = 32
num_episodes = 3500
total_t = 0
rb = ReplayMemory()
episode_rewards = np.zeros(num_episodes)


In [14]:
epsilon = 1.0
epsilon_min = 0.1
epsilon_change = (epsilon - epsilon_min) / 500000

In [15]:
env = gym.envs.make("Breakout-v0")
model = DQN(K=K,conv_layer_sizes=conv_layer_sizes,hidden_layer_sizes=hidden_layer_sizes)
target_model = DQN(K=K,conv_layer_sizes=conv_layer_sizes,hidden_layer_sizes=hidden_layer_sizes)
im_transformer = ImageTransformer()
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [16]:
# populate reolay buffer
obs = env.reset()
for i in range(MIN_EXPERIENCES):
    action = np.random.choice(K)
    obs,reward,done,_ = env.step(action)
    obs_small = im_transformer.transform(obs)
    rb.add_experience(action,obs_small,reward,done)
    if done:
        obs = env.reset()

In [17]:
def running_avg(totalrewards):
    # average results over 100 episodes
    N = len(totalrewards)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = totalrewards[max(0,t-100):(t+1)].mean()
    return running_avg

In [ ]:
t0 = datetime.now()
for i in range(num_episodes):
    total_t, episode_reward, duration, num_steps_in_episode, time_per_step, epsilon = play_one(
        env,
        total_t,
        rb,
        model,
        opt,
        target_model,
        im_transformer,
        gamma,
        batch_sz,
        epsilon,
        epsilon_change,
        epsilon_min,
      )

    episode_rewards[i] = episode_reward

    last_100_avg = episode_rewards[max(0, i - 100):i + 1].mean()
    print("Episode:", i,
        "Duration:", duration,
        "Num steps:", num_steps_in_episode,
        "Reward:", episode_reward,
        "Training time per step:", "%.3f" % time_per_step,
        "Avg Reward (Last 100):", "%.3f" % last_100_avg,
        "Epsilon:", "%.3f" % epsilon
      )
    sys.stdout.flush()
print("Total duration:", datetime.now() - t0)
model.save_weights('model.h5')
# Plot the smoothed returns
y = running_avg(episode_rewards)
plt.plot(episode_rewards, label='orig')
plt.plot(y, label='smoothed')
plt.legend()
plt.show()




Copied model parameters to target network. total_t = 0, period = 10000
Episode: 0 Duration: 0:00:08.882938 Num steps: 176 Reward: 0.0 Training time per step: 0.045 Avg Reward (Last 100): 0.000 Epsilon: 1.000
Episode: 1 Duration: 0:00:07.190657 Num steps: 183 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 0.000 Epsilon: 0.999
Episode: 2 Duration: 0:00:10.567375 Num steps: 262 Reward: 1.0 Training time per step: 0.035 Avg Reward (Last 100): 0.333 Epsilon: 0.999
Episode: 3 Duration: 0:00:07.105578 Num steps: 172 Reward: 0.0 Training time per step: 0.036 Avg Reward (Last 100): 0.250 Epsilon: 0.999
Episode: 4 Duration: 0:00:11.954805 Num steps: 295 Reward: 2.0 Training time per step: 0.036 Avg Reward (Last 100): 0.600 Epsilon: 0.998
Episode: 5 Duration: 0:00:10.080000 Num steps: 257 Reward: 1.0 Training time per step: 0.034 Avg Reward (Last 100): 0.667 Epsilon: 0.998
Episode: 6 Duration: 0:00:10.889602 Num steps: 276 Reward: 2.0 Training time per step: 0.035 Avg Reward (La

Episode: 59 Duration: 0:00:08.800456 Num steps: 203 Reward: 1.0 Training time per step: 0.038 Avg Reward (Last 100): 1.633 Epsilon: 0.971
Episode: 60 Duration: 0:00:11.409208 Num steps: 269 Reward: 2.0 Training time per step: 0.037 Avg Reward (Last 100): 1.639 Epsilon: 0.971
Episode: 61 Duration: 0:00:12.153274 Num steps: 295 Reward: 2.0 Training time per step: 0.036 Avg Reward (Last 100): 1.645 Epsilon: 0.970
Episode: 62 Duration: 0:00:10.588947 Num steps: 270 Reward: 2.0 Training time per step: 0.034 Avg Reward (Last 100): 1.651 Epsilon: 0.970
Episode: 63 Duration: 0:00:11.846984 Num steps: 306 Reward: 3.0 Training time per step: 0.034 Avg Reward (Last 100): 1.672 Epsilon: 0.969
Episode: 64 Duration: 0:00:06.813523 Num steps: 174 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 1.646 Epsilon: 0.969
Episode: 65 Duration: 0:00:09.486357 Num steps: 230 Reward: 1.0 Training time per step: 0.036 Avg Reward (Last 100): 1.636 Epsilon: 0.969
Episode: 66 Duration: 0:00:11.1604

Episode: 118 Duration: 0:00:14.990181 Num steps: 381 Reward: 4.0 Training time per step: 0.034 Avg Reward (Last 100): 1.574 Epsilon: 0.945
Episode: 119 Duration: 0:00:09.310085 Num steps: 236 Reward: 1.0 Training time per step: 0.034 Avg Reward (Last 100): 1.564 Epsilon: 0.944
Episode: 120 Duration: 0:00:07.043348 Num steps: 178 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 1.535 Epsilon: 0.944
Episode: 121 Duration: 0:00:15.467662 Num steps: 389 Reward: 4.0 Training time per step: 0.035 Avg Reward (Last 100): 1.564 Epsilon: 0.943
Episode: 122 Duration: 0:00:06.958900 Num steps: 179 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 1.564 Epsilon: 0.943
Episode: 123 Duration: 0:00:08.433597 Num steps: 211 Reward: 1.0 Training time per step: 0.035 Avg Reward (Last 100): 1.564 Epsilon: 0.943
Episode: 124 Duration: 0:00:09.350227 Num steps: 237 Reward: 1.0 Training time per step: 0.034 Avg Reward (Last 100): 1.574 Epsilon: 0.942
Episode: 125 Duration: 0:00

Episode: 177 Duration: 0:00:09.738998 Num steps: 247 Reward: 2.0 Training time per step: 0.034 Avg Reward (Last 100): 1.366 Epsilon: 0.919
Episode: 178 Duration: 0:00:06.783107 Num steps: 169 Reward: 0.0 Training time per step: 0.035 Avg Reward (Last 100): 1.366 Epsilon: 0.919
Episode: 179 Duration: 0:00:07.194038 Num steps: 177 Reward: 0.0 Training time per step: 0.035 Avg Reward (Last 100): 1.337 Epsilon: 0.918
Episode: 180 Duration: 0:00:11.264668 Num steps: 274 Reward: 2.0 Training time per step: 0.036 Avg Reward (Last 100): 1.356 Epsilon: 0.918
Episode: 181 Duration: 0:00:06.977211 Num steps: 176 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 1.337 Epsilon: 0.917
Episode: 182 Duration: 0:00:06.867669 Num steps: 174 Reward: 0.0 Training time per step: 0.034 Avg Reward (Last 100): 1.317 Epsilon: 0.917
Episode: 183 Duration: 0:00:10.676216 Num steps: 269 Reward: 2.0 Training time per step: 0.034 Avg Reward (Last 100): 1.327 Epsilon: 0.917
Episode: 184 Duration: 0:00

Episode: 235 Duration: 0:00:10.627387 Num steps: 264 Reward: 1.0 Training time per step: 0.035 Avg Reward (Last 100): 1.406 Epsilon: 0.892
Episode: 236 Duration: 0:00:11.498547 Num steps: 287 Reward: 2.0 Training time per step: 0.034 Avg Reward (Last 100): 1.406 Epsilon: 0.891
Episode: 237 Duration: 0:00:12.252604 Num steps: 306 Reward: 2.0 Training time per step: 0.035 Avg Reward (Last 100): 1.406 Epsilon: 0.891
Episode: 238 Duration: 0:00:12.648799 Num steps: 317 Reward: 3.0 Training time per step: 0.034 Avg Reward (Last 100): 1.426 Epsilon: 0.890
Episode: 239 Duration: 0:00:18.561332 Num steps: 470 Reward: 5.0 Training time per step: 0.034 Avg Reward (Last 100): 1.475 Epsilon: 0.889
Episode: 240 Duration: 0:00:09.339780 Num steps: 236 Reward: 1.0 Training time per step: 0.034 Avg Reward (Last 100): 1.475 Epsilon: 0.889
Episode: 241 Duration: 0:00:13.038895 Num steps: 325 Reward: 3.0 Training time per step: 0.035 Avg Reward (Last 100): 1.505 Epsilon: 0.888
Episode: 242 Duration: 0:00

...

Episode: 1690 Duration: 0:00:34.095542 Num steps: 769 Reward: 15.0 Training time per step: 0.035 Avg Reward (Last 100): 10.525 Epsilon: 0.100
Episode: 1691 Duration: 0:00:29.107632 Num steps: 656 Reward: 11.0 Training time per step: 0.035 Avg Reward (Last 100): 10.525 Epsilon: 0.100
Episode: 1692 Duration: 0:00:33.005616 Num steps: 746 Reward: 10.0 Training time per step: 0.035 Avg Reward (Last 100): 10.465 Epsilon: 0.100
Episode: 1693 Duration: 0:00:34.066159 Num steps: 770 Reward: 13.0 Training time per step: 0.035 Avg Reward (Last 100): 10.535 Epsilon: 0.100
Episode: 1694 Duration: 0:00:26.108655 Num steps: 589 Reward: 13.0 Training time per step: 0.035 Avg Reward (Last 100): 10.634 Epsilon: 0.100
Episode: 1695 Duration: 0:00:25.830442 Num steps: 584 Reward: 8.0 Training time per step: 0.035 Avg Reward (Last 100): 10.584 Epsilon: 0.100
Episode: 1696 Duration: 0:00:35.538731 Num steps: 805 Reward: 16.0 Training time per step: 0.035 Avg Reward (Last 100): 10.673 Epsilon: 0.100
Episode

Episode: 1746 Duration: 0:00:30.942846 Num steps: 691 Reward: 10.0 Training time per step: 0.036 Avg Reward (Last 100): 12.267 Epsilon: 0.100
Episode: 1747 Duration: 0:00:25.269015 Num steps: 542 Reward: 7.0 Training time per step: 0.037 Avg Reward (Last 100): 12.238 Epsilon: 0.100
Episode: 1748 Duration: 0:00:35.037932 Num steps: 794 Reward: 13.0 Training time per step: 0.035 Avg Reward (Last 100): 12.238 Epsilon: 0.100
Episode: 1749 Duration: 0:00:34.845219 Num steps: 789 Reward: 16.0 Training time per step: 0.035 Avg Reward (Last 100): 12.307 Epsilon: 0.100
Episode: 1750 Duration: 0:00:31.631103 Num steps: 715 Reward: 14.0 Training time per step: 0.035 Avg Reward (Last 100): 12.337 Epsilon: 0.100
Episode: 1751 Duration: 0:00:28.073386 Num steps: 632 Reward: 9.0 Training time per step: 0.035 Avg Reward (Last 100): 12.317 Epsilon: 0.100
Episode: 1752 Duration: 0:00:40.435371 Num steps: 921 Reward: 17.0 Training time per step: 0.035 Avg Reward (Last 100): 12.406 Epsilon: 0.100
Episode:

Episode: 1802 Duration: 0:00:29.703430 Num steps: 673 Reward: 11.0 Training time per step: 0.035 Avg Reward (Last 100): 12.960 Epsilon: 0.100
Episode: 1803 Duration: 0:00:25.101425 Num steps: 570 Reward: 9.0 Training time per step: 0.035 Avg Reward (Last 100): 12.921 Epsilon: 0.100
Episode: 1804 Duration: 0:00:34.775670 Num steps: 789 Reward: 12.0 Training time per step: 0.035 Avg Reward (Last 100): 12.901 Epsilon: 0.100
Episode: 1805 Duration: 0:00:41.484816 Num steps: 937 Reward: 18.0 Training time per step: 0.035 Avg Reward (Last 100): 12.990 Epsilon: 0.100
Episode: 1806 Duration: 0:00:22.990758 Num steps: 520 Reward: 7.0 Training time per step: 0.035 Avg Reward (Last 100): 12.941 Epsilon: 0.100
Episode: 1807 Duration: 0:00:32.609419 Num steps: 739 Reward: 11.0 Training time per step: 0.035 Avg Reward (Last 100): 12.941 Epsilon: 0.100
Episode: 1808 Duration: 0:00:25.722140 Num steps: 588 Reward: 9.0 Training time per step: 0.035 Avg Reward (Last 100): 12.970 Epsilon: 0.100
Copied mo

Episode: 1858 Duration: 0:00:44.570082 Num steps: 1012 Reward: 24.0 Training time per step: 0.035 Avg Reward (Last 100): 13.881 Epsilon: 0.100
Episode: 1859 Duration: 0:00:31.743000 Num steps: 717 Reward: 16.0 Training time per step: 0.035 Avg Reward (Last 100): 13.911 Epsilon: 0.100
Copied model parameters to target network. total_t = 640000, period = 10000
Episode: 1860 Duration: 0:00:51.611088 Num steps: 1157 Reward: 25.0 Training time per step: 0.036 Avg Reward (Last 100): 14.030 Epsilon: 0.100
Episode: 1861 Duration: 0:00:41.495537 Num steps: 932 Reward: 16.0 Training time per step: 0.036 Avg Reward (Last 100): 14.000 Epsilon: 0.100
Episode: 1862 Duration: 0:00:28.721654 Num steps: 650 Reward: 11.0 Training time per step: 0.035 Avg Reward (Last 100): 14.010 Epsilon: 0.100
Episode: 1863 Duration: 0:00:39.256770 Num steps: 892 Reward: 21.0 Training time per step: 0.035 Avg Reward (Last 100): 14.149 Epsilon: 0.100
Episode: 1864 Duration: 0:00:23.283634 Num steps: 535 Reward: 12.0 Tra

Episode: 1914 Duration: 0:00:40.438697 Num steps: 912 Reward: 19.0 Training time per step: 0.035 Avg Reward (Last 100): 14.703 Epsilon: 0.100
Episode: 1915 Duration: 0:00:32.377309 Num steps: 732 Reward: 16.0 Training time per step: 0.035 Avg Reward (Last 100): 14.703 Epsilon: 0.100
Episode: 1916 Duration: 0:00:40.178702 Num steps: 922 Reward: 18.0 Training time per step: 0.035 Avg Reward (Last 100): 14.733 Epsilon: 0.100
Episode: 1917 Duration: 0:00:29.973401 Num steps: 680 Reward: 10.0 Training time per step: 0.035 Avg Reward (Last 100): 14.703 Epsilon: 0.100
Episode: 1918 Duration: 0:00:33.722213 Num steps: 757 Reward: 14.0 Training time per step: 0.036 Avg Reward (Last 100): 14.713 Epsilon: 0.100
Episode: 1919 Duration: 0:00:47.809766 Num steps: 1081 Reward: 19.0 Training time per step: 0.035 Avg Reward (Last 100): 14.762 Epsilon: 0.100
Episode: 1920 Duration: 0:00:25.606638 Num steps: 588 Reward: 12.0 Training time per step: 0.035 Avg Reward (Last 100): 14.733 Epsilon: 0.100
Copie